<img align="left" src="https://panoptes-uploads.zooniverse.org/project_avatar/86c23ca7-bbaa-4e84-8d8a-876819551431.png" type="image/png" height=100 width=100>
</img>
<h1 align="right">KSO Tutorials #5: Train ML models</h1>
<h3 align="right">Written by the KSO Team</h3>

# 1. Set up and requirements

### Install and import Python packages

In [1]:
from IPython.display import clear_output

try:
    import google.colab
    import os

    IN_COLAB = True
    print("Running in Colab...")

    # Clone repo
    !git clone --recurse-submodules https://github.com/ocean-data-factory-sweden/koster_yolov4.git
    !pip install -q --upgrade pip
    !pip install -qr koster_yolov4/requirements.txt
    !pip install -qr koster_yolov4/yolov5_tracker/requirements.txt

    # Fix libmagic issue
    !apt-get -qq update && apt-get -qq install -y libmagic-dev > /dev/null

    # Replace upsampling script with custom version
    os.chdir("koster_yolov4/tutorials")
    !mv ../src/upsampling.py /usr/local/lib/python3.7/dist-packages/torch/nn/modules/upsampling.py

    # Replace nearest neighbours script with custom version (due to relative path issue)
    !cp ../src/multi_tracker_zoo.py ../yolov5_tracker/trackers/strong_sort/multi_tracker_zoo.py

    # Enable external widgets
    from google.colab import output

    output.enable_custom_widget_manager()

    # Ensure widgets are shown properly
    !jupyter nbextension enable --user --py widgetsnbextension
    !jupyter nbextension enable --user --py jupyter_bbox_widget

    print("All packages are installed and ready to go!")
    try:
        clear_output()
        print("All packages are installed and ready to go!")
    except:
        clear_output()
        print("There have been some issues installing the packages!")
except:
    IN_COLAB = False
    import sys
    import pkgutil

    if pkgutil.find_loader("torch") is None:
        !pip install -q --upgrade pip
        !pip install -q torch==1.8.0 torchvision==0.9.0

    # Replace nearest neighbours script with custom version (due to relative path issue)
    !cp ../src/multi_tracker_zoo.py ../yolov5_tracker/trackers/strong_sort/multi_tracker_zoo.py
    # Ensure widgets are shown properly
    !jupyter nbextension enable --user --py widgetsnbextension
    !jupyter nbextension enable --user --py jupyter_bbox_widget
    clear_output()
    print("Running locally... you're good to go!")

Running locally... you're good to go!


In [2]:
# Set the directory of the libraries
import sys, os

sys.path.append("..")

# Enables testing changes in utils
%load_ext autoreload
%autoreload 2

# Import required modules
from pathlib import Path
from ipyfilechooser import FileChooser
import kso_utils.tutorials_utils as t_utils
import kso_utils.project_utils as p_utils
import kso_utils.server_utils as s_utils
import kso_utils.t5_utils as t5
import wandb

clear_output()
print("Packages loaded successfully")

# Select the model type for training
model_type = t5.choose_model_type()

Packages loaded successfully


Dropdown(description='Required model type:', layout=Layout(width='max-content'), options=(('Object Detection (…

In [3]:
# Model-specific imports
if model_type.value == 1:
    import yolov5.train as train
    import yolov5.detect as detect
    import yolov5.val as val

    print("Object detection model loaded")
elif model_type.value == 2:
    import yolov5.classify.train as train
    import yolov5.classify.predict as detect
    import yolov5.classify.val as val

    print("Image classification model loaded")
elif model_type.value == 3:
    import yolov5.segment.train as train
    import yolov5.segment.predict as detect
    import yolov5.segment.val as val

    print("Image segmentation model loaded")
else:
    print("Invalid model specification")

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: jannesg (koster). Use `wandb login --relogin` to force relogin


Image classification model loaded


# 2. Train the model

🔴 <span style="color:red">&nbsp;NOTE: To be able to train your own models, you will need access to the Koster WANDB group. You may request this access by contacting jurie.germishuys@combine.se. </span>

### Choose your project

In [4]:
project_name = t_utils.choose_project()

Dropdown(description='Project:', options=('Template project', 'Koster_Seafloor_Obs', 'Spyfish_Aotearoa', 'SGU'…

In [5]:
project = p_utils.find_project(project_name=project_name.value)

INFO:root:SGU loaded succesfully
SGU loaded succesfully


In [6]:
# Only for Template Project (downloading prepared data)
s_utils.get_ml_data(project)

INFO:root:No download method implemented for this data
No download method implemented for this data


### Configure data paths

In [50]:
# Specify path containing the images and labels folders.
output_folder = t_utils.choose_folder(
    project.photo_folder if not project.photo_folder == "None" else ".", "output"
)

/opt/conda/lib/python3.8/site-packages/tqdm/std.py:725: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


FileChooser(path='/mimer/NOBACKUP/groups/snic2022-22-1210/KSO_SGU_pz/sgu_samples', filename='', title='HTML(va…

🔴 <span style="color:red">&nbsp;NOTE: Each model type requires a specific folder structure to be in place. To be able to train your own Object Detection models, your data_path must contain a yml file for data and hyperparameters. See https://github.com/ultralytics/yolov5/wiki/Train-Custom-Data#11-create-datasetyaml. For image classification models, there should be 3 folders (train, val, test) each containing images in class_name folders. For segmentation models, polygon coordinates are also required. </span>

In [54]:
# Fix important paths
data_path, hyps_path = t5.setup_paths(output_folder.selected, model_type.value)
project_path = str(Path(output_folder.selected, project.Project_name.lower()))

INFO:root:Paths do not need to be changed for this model type.
Paths do not need to be changed for this model type.


### Choose a suitable experiment name

In [9]:
exp_name = t5.choose_experiment_name()

Text(value='exp_name', description='Experiment name:', placeholder='Choose an experiment name', style=Descript…

### Choose model to use for training

In [51]:
# Specify path to download baseline model
download_folder = t_utils.choose_folder(
    project.photo_folder if not project.photo_folder == "None" else ".",
    "model download",
)

FileChooser(path='/mimer/NOBACKUP/groups/snic2022-22-1210/KSO_SGU_pz/sgu_samples', filename='', title='HTML(va…

In [55]:
weights = t5.choose_baseline_model(download_folder.selected)

Dropdown(description='Select model:', layout=Layout(width='50%'), options=(('yolov5m-classifier', <ArtifactCol…

Output()

### Train model with given configuration

In [12]:
batch_size, epochs = t5.choose_train_params()

In [57]:
if model_type.value == 1:
    train.run(
        entity="koster",
        data=data_path,
        hyp=hyps_path,
        weights=weights.artifact_path,
        project=os.path.basename(project_path).replace(" ", "_").lower(),
        name=exp_name.value,
        img_size=[720, 540],
        batch_size=int(batch_size.value),
        epochs=epochs.value,
        workers=1,
        single_cls=False,
        cache_images=True,
    )
elif model_type.value == 2:
    train.run(
        entity="koster",
        data=data_path,
        model=weights.artifact_path,
        project=os.path.basename(project_path).replace(" ", "_").lower(),
        name=exp_name.value,
        imgsz=224,
        batch_size=int(batch_size.value),
        epochs=10,
        workers=4,
    )
else:
    print("Segmentation model training not yet supported.")

INFO:yolov5:classify/train: model=/mimer/NOBACKUP/groups/snic2021-6-9/tmp_dir/sgu_out_split_224_0.8_1337/yolov5m-cls.pt, data=/mimer/NOBACKUP/groups/snic2021-6-9/tmp_dir/sgu_out_split_224_0.8_1337/, epochs=10, batch_size=32, imgsz=224, nosave=False, cache=None, device=, workers=4, project=sgu, name=classification_model, exist_ok=False, pretrained=True, optimizer=Adam, lr0=0.001, decay=5e-05, label_smoothing=0.1, cutoff=None, dropout=None, verbose=False, seed=0, local_rank=-1, entity=koster
classify/train: model=/mimer/NOBACKUP/groups/snic2021-6-9/tmp_dir/sgu_out_split_224_0.8_1337/yolov5m-cls.pt, data=/mimer/NOBACKUP/groups/snic2021-6-9/tmp_dir/sgu_out_split_224_0.8_1337/, epochs=10, batch_size=32, imgsz=224, nosave=False, cache=None, device=, workers=4, project=sgu, name=classification_model, exist_ok=False, pretrained=True, optimizer=Adam, lr0=0.001, decay=5e-05, label_smoothing=0.1, cutoff=None, dropout=None, verbose=False, seed=0, local_rank=-1, entity=koster
INFO:yolov5:github: ⚠️

INFO:yolov5:Model summary: 212 layers, 11683045 parameters, 11683045 gradients, 30.9 GFLOPs
Model summary: 212 layers, 11683045 parameters, 11683045 gradients, 30.9 GFLOPs
INFO:yolov5:optimizer: Adam(lr=0.001) with parameter groups 46 weight(decay=0.0), 47 weight(decay=5e-05), 47 bias
optimizer: Adam(lr=0.001) with parameter groups 46 weight(decay=0.0), 47 weight(decay=5e-05), 47 bias
INFO:yolov5:Image sizes 224 train, 224 test
Using 4 dataloader workers
Logging results to sgu/classification_model11
Starting /mimer/NOBACKUP/groups/snic2021-6-9/tmp_dir/sgu_out_split_224_0.8_1337/yolov5m-cls.pt training on /mimer/NOBACKUP/groups/snic2021-6-9/tmp_dir/sgu_out_split_224_0.8_1337 dataset with 5 classes for 10 epochs...

     Epoch   GPU_mem  train_loss   test_loss    top1_acc    top5_acc
Image sizes 224 train, 224 test
Using 4 dataloader workers
Logging results to sgu/classification_model11
Starting /mimer/NOBACKUP/groups/snic2021-6-9/tmp_dir/sgu_out_split_224_0.8_1337/yolov5m-cls.pt trainin

# 3. Evaluate model performance

In [ ]:
conf_thres = t5.choose_eval_params()

In [ ]:
# Choose model
eval_model = FileChooser(project_path)
display(eval_model)

In [ ]:
# Find trained model weights
tuned_weights = f"{Path(project_path, eval_model.selected, 'weights', 'best.pt')}"

In [ ]:
# Evaluate YOLO Model on Unseen Test data
val.run(
    data=data_path,
    weights=tuned_weights,
    conf_thres=conf_thres.value,
    imgsz=640 if model_type.value == 1 else 224,
    half=False,
)

# (Optional) : 4. Enhance annotations using trained model

Enhancement uses the trained model to increase the amount of annotations in the training data. This should only be done in cases where it is absolutely necessary as bad predictions lead to worse predictions when used to train the next iteration of the model. 


🔴 <span style="color:red">&nbsp;NOTE: We recommend using a relatively high confidence threshold when enhancing trained models as low confidence predictions could significantly impact the quality of your annotated data. This is currently only available for object detection models.  </span>

In [ ]:
eh_conf_thres = t5.choose_eval_params()

In [ ]:
if model_type.value == 1:
    detect.run(
        weights=tuned_weights,
        source=output_folder.selected + "/images",
        imgsz=[640, 640],
        conf_thres=eh_conf_thres.value,
        save_txt=True,
    )
elif model_type.value == 2:
    print("Enhancements not supported for image classification models at this time.")
else:
    print("Enhancements not supported for segmentation models at this time.")

### Choose run to use as enhanced annotations

In [ ]:
runs = FileChooser(".")
display(runs)

In [ ]:
if model_type.value == 1:
    !mv {output_folder}"/labels" {output_folder}"/labels_org"
    !mv {runs.selected}"/labels" {output_folder}"/labels"

#### Once you have moved the new labels to the original label location, you can return to Step 2 and train your model again. 

🔴 <span style="color:red">&nbsp;NOTE: Run this cell to complete WANDB run, OR else artifacts will not be shown.

In [56]:
wandb.finish()

lr/0,█▇▅▄▂▁
metrics/accuracy_top1,▁▂▆▅██
metrics/accuracy_top5,▁▁▁▁▁▁
test/loss,█▇▃▄▁▁
train/loss,█▅▄▃▂▁
lr/0,0.00041
metrics/accuracy_top1,0.69013
metrics/accuracy_top5,1.0
test/loss,0.76902
train/loss,0.74242


In [ ]:
# END